In [1]:
import numpy
import os
import glob
import matplotlib.pyplot as plt
from skimage import transform as tf
import math 
from skimage.transform import rescale
%matplotlib inline

import sys
sys.path.append('/vol/programs/keras-extras/utils/')

/home/ubuntu/miniconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
import keras

from keras.utils import np_utils

from keras.models import Model
from keras.layers import Input, merge, Convolution2D, MaxPooling2D, UpSampling2D
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as K

# from multi_gpu import *

Using TensorFlow backend.


In [3]:
dataLocation = '/vol/data/numpyTraining/'
trainingImages = 'padded_training_images.npy' #train_images.npy'
trainingLabels = 'padded_training_labels.npy' #'train_labels.npy'

img_rows = 320   
img_cols = 256   

smooth = 1.

In [4]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [32]:
def get_unet():
    inputs = Input((1, img_rows, img_cols))
    conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(inputs)
    conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(pool1)
    conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(pool2)
    conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(pool3)
    conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(pool4)
    conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(conv5)

    up6 = merge([UpSampling2D(size=(2, 2))(conv5), conv4], mode='concat', concat_axis=1)
    
    conv6 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(up6)
    conv6 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conv6)

    up7 = merge([UpSampling2D(size=(2, 2))(conv6), conv3], mode='concat', concat_axis=1)
    conv7 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(up7)
    conv7 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv7)

    up8 = merge([UpSampling2D(size=(2, 2))(conv7), conv2], mode='concat', concat_axis=1)
    conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(up8)
    conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv8)

    up9 = merge([UpSampling2D(size=(2, 2))(conv8), conv1], mode='concat', concat_axis=1)
    conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(up9)
    conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv9)

    conv10 = Convolution2D(1, 1, 1, activation='sigmoid')(conv9)

    model = Model(input=inputs, output=conv10)
    model.compile(optimizer=Adam(lr=1e-4), loss=dice_coef_loss, metrics=[dice_coef])
    #learning rate (lr) was 1e-5
    
    

    return model 

In [6]:
def preprocess(labels):
    labels[numpy.where(labels==1)] = 0
    labels[numpy.where(labels==3)] = 0
    labels[numpy.where(labels==2)] = 1
    labels[numpy.where(labels==4)] = 1
    return labels

In [7]:
def import_and_process(trainingImagesLocation, testingImagesLocation):
    print('-'*30)
    print('Loading and preprocessing train data...')
    print('-'*30)
    trainImages = (numpy.load(trainingImagesLocation)).astype(numpy.float32)
    trainLabels = (numpy.load(testingImagesLocation)).astype(numpy.int8)
    trainLabels = preprocess(trainLabels)
    return(trainLabels, trainImages)

In [8]:
trainLabels, trainImages = import_and_process(dataLocation + trainingImages, dataLocation + trainingLabels)

------------------------------
Loading and preprocessing train data...
------------------------------


In [ ]:
# trainLabelsShape = trainLabels.shape
# trainImagesShape = trainImages.shape
# trainLabels = tf.resize(trainLabels, (trainLabelsShape[0], trainLabelsShape[1]/2, trainLabelsShape[2]/2))
# trainImages = tf.resize(trainImages, (trainImagesShape[0], trainImagesShape[1]/2, trainImagesShape[2]/2))

In [9]:
trainLabels = numpy.expand_dims(trainLabels, axis=1)
trainImages = numpy.expand_dims(trainImages, axis=1)

In [18]:
trainImages.astype(numpy.float16)

array([[[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         ..., 
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]]],


       [[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         ..., 
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]]],


       [[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         ..., 
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]]],


       ..., 
       [[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [

In [25]:
print('Shape of Labels: ' + str(trainLabels.shape))
print('Shape of Images: ' + str(trainImages.shape))

Shape of Labels: (8715, 1, 320, 256)
Shape of Images: (8715, 1, 320, 256)


# Parameters
Tried batch of 24 and it was running out of memory. 12 didnt run out of memory and was slightly faster.
batches of 12 lead to ~1hr/epoch processing time (just under & 3500+ s/epoch). 

Am trying batches of 16 to see if memory is still fine. Technically the higher the batch size will decrease training time, as long as still in memory. 
Also going to try and increase the learning rate of the optimizer. 
Scratch that... I reduced the pixel type to int 16 which should half the memory of the inputs. So, I should in theory be able to double the batch size. I have increased the batch size to 30 and will use that instead of 16. I have also increased the learning rate form 1e-5 to 1e-3.... that should also make convergence quicker... hopefully. 

In [33]:
def train(dataLocation):
    print('-'*30)
    print('Creating and compiling model...')
    print('-'*30)
    model = get_unet()
    model_checkpoint = ModelCheckpoint(dataLocation + 'uNet.hdf5', monitor='loss', save_best_only=True)
    
    print('-'*30)
    print('Fitting model...')
    print('-'*30)
    model.fit(trainImages, trainLabels, batch_size=24, nb_epoch=10, verbose=1, shuffle=True,
              callbacks=[model_checkpoint]) 
    
    

In [34]:
modelTest = get_unet()
print(modelTest.summary())


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_9 (InputLayer)             (None, 1, 320, 256)   0                                            
____________________________________________________________________________________________________
convolution2d_153 (Convolution2D)(None, 32, 320, 256)  320         input_9[0][0]                    
____________________________________________________________________________________________________
convolution2d_154 (Convolution2D)(None, 32, 320, 256)  9248        convolution2d_153[0][0]          
____________________________________________________________________________________________________
maxpooling2d_33 (MaxPooling2D)   (None, 32, 160, 128)  0           convolution2d_154[0][0]          
___________________________________________________________________________________________

In [ ]:
train(dataLocation)

------------------------------
Creating and compiling model...
------------------------------
------------------------------
Fitting model...
------------------------------
Epoch 1/10
5136/8715 [================>.............] - ETA: 1467s - loss: -0.0047 - dice_coef: 0.0047

# Below are the results from the first full pass on Jan.31.2017. 
I believe that there may have been an issue with the order of the segmentations/images. This is the first thing I am going to explore. 

## Creating and compiling model...

### Fitting Model 

Epoch 1/10
8715/8715 [==============================] - 3537s - loss: -0.1452 - dice_coef: 0.1452     
Epoch 2/10
8715/8715 [==============================] - 3518s - loss: -0.1795 - dice_coef: 0.1795     
Epoch 3/10
8715/8715 [==============================] - 3518s - loss: -0.1853 - dice_coef: 0.1853     
Epoch 4/10
8715/8715 [==============================] - 3518s - loss: -0.1923 - dice_coef: 0.1923     
Epoch 5/10
8715/8715 [==============================] - 3517s - loss: -0.1962 - dice_coef: 0.1962     
Epoch 6/10
8715/8715 [==============================] - 3519s - loss: -0.1991 - dice_coef: 0.1991     
Epoch 7/10
8715/8715 [==============================] - 3529s - loss: -0.2050 - dice_coef: 0.2050     
Epoch 8/10
8715/8715 [==============================] - 3523s - loss: -0.2100 - dice_coef: 0.2100     
Epoch 9/10
8715/8715 [==============================] - 3521s - loss: -0.2147 - dice_coef: 0.2147     
Epoch 10/10
8715/8715 [==============================] - 3519s - loss: -0.2208 - dice_coef: 0.2208 
